### The Problem
In a simplistic architecture, the request from a client goes to any of the $N$ servers using the simple calculation $hash(request)\ \%\ N$. There are many reasons why we would want same request (request with similar characteristics) to go to the same server. The server may be holding some cache that corresponds to the resource key in request. The server may be storing the session corresponding to the request.So it is clear that if we add a new server or remove a new server we cannot change our distribution calculation to $hash(request)\ \%\ (N+1)$.  

We can also look it in terms of database. Suppose we have $N$ database servers, each storing key-value pair of information. Which server stores which key-value pair is again calculated by $hash(key)\ \%\ N$ .  
![db](https://i.imgur.com/Xn2Loly.png)  

If second database dies then there will be a lot of redistribution of data. 100% of the databases need to be updated.
![db removed](https://i.imgur.com/5zXG1Gd.png)

Similarly, if we added a new database,
![db added](https://i.imgur.com/lJ0vAdZ.png)

### Consistent Hashing
Consistent hashing facilitates the distribution of data across a set of nodes in such a way that minimizes the re-mapping/ reorganization of data when nodes are added or removed.  

**Hash Function:** we take a hash function which distributes hash across a large space, $hash(x) = y$, where $y$ is a large number.  
**Hash Ring:** the hash value is represented in form of ring as shown below:  
![ring](https://i.imgur.com/eFQc813.png)

Next we have to place our database servers in the ring. So we hash the databases using the choosen hashing function. $hash(ip\_address)$.
![db ring](https://i.imgur.com/sFaNJDJ.png)

One a query for a key (or a key needs to be inserted), the same hashing function calculates the hash for that key and it is placed on the ring. The first database after the key hash found in counter clock direction is the database corresponding to the key.  
![key ring](https://i.imgur.com/JEDPCtC.png)  

If a database is removed, then only the keys corresponding to the removed server need to be updated.
![removed](https://i.imgur.com/iNNv4h1.png)

If the database server is added,
![added](https://i.imgur.com/qpYfnnE.png)

### Add Nodes
The problem with the above approach is that distribution of the database servers on the ring may be very skewed. For example:  
![non-uniform](https://i.imgur.com/ztlNuZ1.png)
Database server 1 gets the most load. To make sure that all servers get roughly the same load, we have to increase the number of servers. So we introduce virtual database servers. Each server has equal number of copies of the database server.  
![virtual nodes](https://i.imgur.com/nmzT6gG.png)

Now whenever we remove or add database servers, load gets distributed roughly equally among all other servers.

### Implementation
Server class:

In [1]:
class Server:
    def __init__(self, ip):
        self.ip = ip

In [ ]:
import md5
class HashRing:
    def __init__(self, replicas=100):
        self.replicas = replicas
        
        # Storage for servers
        self.nodes = {}
        
        # Storage for keys
        self.keys = []
        
    def hash_key(self, key):
        return long(md5.md5(key).hexdigest, 16)
    
    def replica_hashes(self, server_ip):
        return (self.hash_key(server_ip + i) for i in range(replicas))
    
    def __setitem__(self, server_ip, server):
        for h in self.replica_hashes(server_ip):
            # Check if server is already present in the ring
            if h in self.nodes:
                raise ValueError('Server already added')
            
            self.nodes[h] = server
            
    def __getitem__(self, key):
        # Get hash for the key
        h = self.hash_key(key)